That is draft raw dirty code

In [13]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch.nn.functional as F

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B").to(device)

In [3]:
prompt = 'Who are you?'
messages = [
    {"role": "user", "content": prompt},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

<think>
Okay, the user asked, "Who are you?" I need to respond in a way that's friendly and helpful. Let me start by acknowledging their question. I should mention my name and


In [4]:
# multiple prompts case
prompts = ['Who are you?', 'Will AI make humanity slave?']
messages = [
    [{"role": "user", "content": prompt}]
    for prompt in prompts
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
    padding=True,
    padding_side='left'
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
for i, output in enumerate(outputs):
    print(f'{i}-th generation: {tokenizer.decode(output[inputs["input_ids"].shape[-1]:])}')

0-th generation: <think>
Okay, the user asked, "Who are you?" I need to respond appropriately. First, I should acknowledge their question. Then, I should explain that I'm an AI assistant designed to
1-th generation: <think>
Okay, so the user is asking if AI will make humanity slave. Hmm, that's a profound question. Let me start by breaking it down. First, I need to address the core


In [5]:
print(tokenizer(prompt))
print(inputs)
print(tokenizer.decode(inputs['input_ids'][0]))

{'input_ids': [15191, 525, 498, 30], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': tensor([[151643, 151643, 151644,    872,    198,  15191,    525,    498,     30,
         151645,    198, 151644,  77091,    198],
        [151644,    872,    198,   9945,  15235,   1281,  21941,  20362,     30,
         151645,    198, 151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
<|endoftext|><|endoftext|><|im_start|>user
Who are you?<|im_end|>
<|im_start|>assistant



In [6]:
print('qwen config')
model.config.model_type

qwen config


'qwen3'

In [ ]:
# Try to parse generated scores
gen_cfg = GenerationConfig(num_beams=1,
                            do_sample=False,
                            return_dict_in_generate=True,
                            # output_scores=True,
                            output_logits=True)

prompts = ['Hello!', 'Tell me about Stepan Razin']

messages = [
    [{"role": "user", "content": prompt}]
    for prompt in prompts
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
    padding=True,
    padding_side='left',
    enable_thinking=False
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=10, generation_config=gen_cfg)

`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'pad_token_id': 151643, 'bos_token_id': 151643, 'eos_token_id': [151645, 151643]}. If this is not desired, please set these values explicitly.


In [64]:
for seq in outputs['sequences']:
    print(tokenizer.decode(seq))

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
<think>

</think>

Hello! How can I assist you today?<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

I want get list of only generated indexes and corresponding scores for each generated tokens.

In [65]:
for seq in outputs['sequences']:
    print(tokenizer.decode(seq))

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
<think>

</think>

Hello! How can I assist you today?<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [25]:
def truncate_output(outputs, input_ids, tokenizer):
    trunc_seq_list = []

    for i, seq in enumerate(outputs['sequences']):
        first_idx = len(input_ids[i])
        last_idx = torch.nonzero(seq == tokenizer.eos_token_id)[-1]
        trunc_seq = seq[first_idx : last_idx + 1]

        trunc_seq_list.append(trunc_seq)
    scores_list = [[] for _ in range(len(trunc_seq_list))]

    for step, scores in enumerate(outputs['scores']):
        for i, seq in enumerate(trunc_seq_list):
            if step >= len(seq):
                continue

            scores_list[i].append(scores[scores != float('-inf')][i].item())

    scores_tensors_list = []
    for trunc_seq, scores in zip(trunc_seq_list, scores_list):
        assert len(trunc_seq_list) == len(scores_list)
        scores_tensors_list.append(torch.tensor(scores))

    return trunc_seq_list, scores_tensors_list

In [50]:
def truncate_output(outputs, input_ids, tokenizer, return_log_prob=True):
    trunc_seq_list = []

    for i, seq in enumerate(outputs['sequences']):
        first_idx = len(input_ids[i])
        if seq[-1] == tokenizer.eos_token_id or seq[-1] == tokenizer.pad_token_id:
            last_idx = torch.nonzero(seq == tokenizer.eos_token_id)[-1]
        else:
            last_idx = len(seq)
        trunc_seq = seq[first_idx : last_idx + 1]

        trunc_seq_list.append(trunc_seq)

    result_list = [[] for _ in range(len(trunc_seq_list))]
    for step, logits in enumerate(outputs['logits']):
        for i, seq in enumerate(trunc_seq_list):
            if step >= len(seq):
                continue

            if return_log_prob:
                result_list[i].append(F.log_softmax(logits[i], dim=0)[seq[step]])
            else:
                result_list[i].append(logits[i])

    if not return_log_prob:
        return trunc_seq_list, result_list

    result_tensors_list = []
    for trunc_seq, scores in zip(trunc_seq_list, result_list):
        assert len(trunc_seq) == len(scores)
        result_tensors_list.append(torch.tensor(scores))

    return trunc_seq_list, result_tensors_list

In [55]:
trunc_seq_list, scores_list = truncate_output(outputs, inputs['input_ids'], tokenizer)

In [58]:
scores_list[0].exp()

tensor([0.9989, 0.9987, 0.7887, 0.9995, 0.9995, 0.9483, 0.9999, 0.9721, 0.9997,
        0.8988])

In [66]:
trunc_seq_list = []

for seq in outputs['sequences']:
    first_idx = len(inputs['input_ids'][0])
    last_idx = torch.nonzero(seq == tokenizer.eos_token_id)[-1]
    trunc_seq = seq[first_idx : last_idx + 1]

    trunc_seq_list.append(trunc_seq)

In [102]:
scores_list = [[] for _ in range(len(trunc_seq_list))]

for step, scores in enumerate(outputs['scores']):
    for i, seq in enumerate(trunc_seq_list):
        if step >= len(seq):
            continue

        scores_list[i].append(scores[scores != float('-inf')][i])

In [81]:
outputs['scores'][0][outputs['scores'][0] != float('-inf')][0]

tensor(52.2759, device='cuda:0')

In [14]:
one_step_scores = outputs['scores'][0][0]
print(one_step_scores[one_step_scores != float('-inf')])

tensor([51.7064], device='cuda:0')
